In [31]:
import numpy as np
import wavload
import librosa
import math        
import pyaudio     
import random
import tflearn
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [32]:
"""Making samples - Mark 2 """

fs = 22050      # sampling rate, Hz, must be integer
duration = 5   # in seconds, may be float

num_samples = 10000
freq = np.zeros((num_samples,10))
amp = np.zeros((num_samples,10))
X_data = np.zeros((num_samples,fs*duration))


for i in range (0,num_samples):
    freq1 = random.uniform(16,2000)        # sine frequency, Hz, may be float
    freq2 = 2*freq1
    freq3 = 3*freq1
    freq4 = 4*freq1
    freq5 = 5*freq1
    freq6 = 6*freq1
    freq7 = 7*freq1
    freq8 = 8*freq1
    freq9 = 9*freq1
    freq10 = 10*freq1

    amp1 = random.uniform(0,1)     # range [0.0, 1.0]
    amp2 = random.uniform(0,amp1)
    amp3 = random.uniform(0,amp2)
    amp4 = random.uniform(0,amp3)
    amp5 = random.uniform(0,amp4)
    amp6 = random.uniform(0,amp5)
    amp7 = random.uniform(0,amp6)
    amp8 = random.uniform(0,amp7)
    amp9 = random.uniform(0,amp8)
    amp10 = random.uniform(0,amp9)


    # generate samples, note conversion to float32 array
    sample1 = (amp1*(np.sin(2*np.pi*np.arange(fs*duration)*freq1/fs))).astype(np.float32)
    sample2 = (amp2*(np.sin(2*np.pi*np.arange(fs*duration)*freq2/fs))).astype(np.float32)
    sample3 = (amp3*(np.sin(2*np.pi*np.arange(fs*duration)*freq3/fs))).astype(np.float32)
    sample4 = (amp4*(np.sin(2*np.pi*np.arange(fs*duration)*freq4/fs))).astype(np.float32)
    sample5 = (amp5*(np.sin(2*np.pi*np.arange(fs*duration)*freq5/fs))).astype(np.float32)
    sample6 = (amp6*(np.sin(2*np.pi*np.arange(fs*duration)*freq6/fs))).astype(np.float32)
    sample7 = (amp7*(np.sin(2*np.pi*np.arange(fs*duration)*freq7/fs))).astype(np.float32)
    sample8 = (amp8*(np.sin(2*np.pi*np.arange(fs*duration)*freq8/fs))).astype(np.float32)
    sample9 = (amp9*(np.sin(2*np.pi*np.arange(fs*duration)*freq9/fs))).astype(np.float32)
    sample10 = (amp10*(np.sin(2*np.pi*np.arange(fs*duration)*freq10/fs))).astype(np.float32)

    audio = ((sample1
                 +sample2
                 +sample3
                 +sample4
                 +sample5
                 +sample6
                 +sample7
                 +sample8
                 +sample9
                 +sample10)/10)

    freq[i] = (freq1,freq2,freq3,freq4,freq5,freq6,freq7,freq8,freq9,freq10)    
    amp[i] = (amp1,amp2,amp3,amp4,amp5,amp6,amp7,amp8,amp9,amp10)
    X_data[i] = audio
    #print(audio.shape)
    
# for paFloat32 sample values must be in range [-1.0, 1.0]
#stream = p.open(format=pyaudio.paFloat32,
#                    channels=1,
#                    rate=fs,
#                    output=True)

# play. May repeat with different volume values (if done interactively) 
#stream.write(audio)

#stream.stop_stream()
#stream.close()

#p.terminate()


In [35]:
"""Taking MFCC of audio data"""
MFCC = np.zeros((num_samples,20,216))
for i in range (0,num_samples):
    MFCC[i] = librosa.feature.mfcc(y=X_data[i], sr=fs, n_mfcc=20)
print(MFCC.shape)

(10000, 20, 216)


In [36]:
"""Loading parameters as Y data"""
Y_data = np.concatenate((freq , amp), axis=1)
print(Y_data.shape)

(10000, 20)


In [37]:
"""Defining important variables for LSTM"""

features_cols = MFCC[0].shape[0]
features_rows = 1
parameter_size = Y_data[0].shape[0]
print(parameter_size)

20


In [22]:
"""LSTM network"""

net = tflearn.input_data([None, features_cols, features_rows])
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, parameter_size, activation='relu')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='mean_square')
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(train_x, train_y, validation_set=(test_x, test_y), show_metric=True,
          batch_size=32, n_epoch=1)

W0912 21:22:00.766973 139620787279680 deprecation_wrapper.py:119] From /home/josephcleveland/Research/venv/lib/python3.6/site-packages/tflearn/layers/core.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0912 21:22:00.771616 139620787279680 deprecation.py:323] From /home/josephcleveland/Research/venv/lib/python3.6/site-packages/tflearn/layers/recurrent.py:69: static_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
W0912 21:22:00.788044 139620787279680 deprecation.py:506] From /home/josephcleveland/Research/venv/lib/python3.6/site-packages/tflearn/layers/recurrent.py:681: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - k

KeyboardInterrupt: 

In [7]:
#X_data = np.expand_dims(X_data, axis = 2)

In [38]:
model = Sequential()
model.add(LSTM(128))
model.add(Dense(units=20, activation='relu'))
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(MFCC, Y_data)

10000/10000 [==============================] - 6s 598us/sample - loss: 9665878.4044 - acc: 0.9968


In [39]:
outs = model.predict(MFCC)

In [42]:
outs[4]

array([8.6155450e+02, 1.7232880e+03, 2.5850974e+03, 3.4463672e+03,
       4.3091665e+03, 5.1688496e+03, 6.0312080e+03, 6.8897549e+03,
       7.7516074e+03, 8.6209453e+03, 6.1200924e-02, 3.6561254e-01,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       2.5388947e-01, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
      dtype=float32)

In [43]:
Y_data[4]

array([4.25396355e+02, 8.50792710e+02, 1.27618906e+03, 1.70158542e+03,
       2.12698177e+03, 2.55237813e+03, 2.97777448e+03, 3.40317084e+03,
       3.82856719e+03, 4.25396355e+03, 2.64696885e-01, 1.54517722e-01,
       1.00156167e-01, 2.29255772e-02, 1.23416455e-02, 7.02777104e-03,
       3.10745778e-03, 2.44213197e-03, 9.97888219e-04, 7.17068789e-05])